### Deploy Trained Model - Azure Machine Learning Service

In [47]:
import azureml
from azureml.core import Workspace, Run

In [48]:
ws = Workspace.from_config()

Found the config file in: /Users/Rithin/Desktop/Projects/session-based-vehicle-recommendations/model/config.json


In [49]:
output_folder = "outputs"
model_name = "vehicle_recommendations_model"
item_mapping_name = f'{model_name}_item_veh_mapping'
veh_mapping_name = f'{model_name}_veh_item_mapping'

In [8]:
from azureml.core.model import Model

model_gnn = Model.register(model_path=f'{output_folder}/{model_name}.pt',
                        model_name=model_name,
                        tags={"data": "vehicle_clicks", "model": "recommendation"},
                        description="Session Based Vehicle Recommendations using GNN",
                        workspace=ws)

model_item_mappings = Model.register(model_path=f'{output_folder}/{item_mapping_name}.dat',
                        model_name='item_to_veh_mappings',
                        tags={"data": "vehicle_clicks", "model": "recommendation", "model_name":model_name},
                        description=f'Item to Vehicle Mappings for model {model_name}',
                        workspace=ws)

model_veh_mappings = Model.register(model_path=f'{output_folder}/{veh_mapping_name}.dat',
                        model_name='veh_to_item_mappings',
                        tags={"data": "vehicle_clicks", "model": "recommendation", "model_name":model_name},
                        description=f'Vehicle to Item Mappings for model {model_name}',
                        workspace=ws)

Registering model vehicle_recommendations_model
Registering model item_to_veh_mappings
Registering model veh_to_item_mappings


### Create Environment File

In [50]:
from azureml.core.conda_dependencies import CondaDependencies 

myenv = CondaDependencies()
myenv.add_pip_package("torch")

with open("myenv.yml","w") as f:
    f.write(myenv.serialize_to_string())


### Create Congifuration File

In [51]:
from azureml.core.webservice import AciWebservice

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               tags={"data": "Vehicle_Recommendations",  "method" : "GNN"}, 
                                               description='Session based vehicle recommendations using GNN')

In [52]:
model1=Model(ws, 'vehicle_recommendations_model')
model2=Model(ws, 'item_to_veh_mappings')
model3=Model(ws, 'veh_to_item_mappings')

In [53]:
%%time
from azureml.core.webservice import Webservice
from azureml.core.image import ContainerImage

# configure the image
image_config = ContainerImage.image_configuration(execution_script="score.py", 
                                                  runtime="python", 
                                                  conda_file="myenv.yml")

service = Webservice.deploy_from_model(workspace=ws,
                                       name='veh-recommendations-init',
                                       deployment_config=aciconfig,
                                       models=[model1, model2, model3],
                                       image_config=image_config)

#service.wait_for_deployment(show_output=True)

Creating image
Image creation operation finished for image veh-recommendations-init:4, operation "Succeeded"
Creating service
CPU times: user 1.3 s, sys: 102 ms, total: 1.4 s
Wall time: 3min 33s


In [ ]:
print(service.scoring_uri)